# Notæ 1
## Modelado y resolución de PLE en Python

### Introducción

El objetivo de este primer notebook introductorio es aprender a escribir modelos de Programación Lineal Entera (PLE) en Python y resolverlos por medio de CPLEX. 

Vamos a seguir el <a href="https://en.wikipedia.org/wiki/Eight_queens_puzzle">Problema de las N Reinas</a> como caso de estudio, en su versión de problema de optimizaión. 

#### Problema de las $N$ reinas
<i>Entrada:</i> Un número natural $N \geq 4$.<br>
<i>Objetivo:</i> Ubicar la mayor cantidad de reinas en las casillas de un tablero de ajedrez de tamaño $N \times N$ inicialmente vacío, de modo que ningún par de reinas se ataque mutuamente (recordar que dos reinas se atacan si se encuentran en una misma fila, columna o diagonal).

Python dispone de un paquete llamado <code>chess</code> que nos permite fácilmente graficar tableros y piezas de ajedrez (si... python tiene infinidad de cosas, algunas más útiles que otras &#128514;). Este paquete será útil para mostrar un ejemplo de colocación. En este caso, se coloca una reina en la casilla "c6" y se muestran en rojo las casillas atacadas. No se preocupen por entender el código de abajo, simplemente ejecútenlo y vean en resultado. Obviamente mientras sea posible, deberemos seguir colocando reinas en las casillas restantes. Más adelante volveremos a usar este paquete para graficar soluciones del problema.

In [ ]:
import chess
import chess.svg

board = chess.Board()
board.clear()
board.set_piece_at(chess.C6, chess.Piece(chess.QUEEN, chess.WHITE))
chess.svg.board(board, fill=dict.fromkeys(board.attacks(chess.C6), "#cc0000cc"), size=350)

### Formulación de PLE

Una forma de resolver este problema es modelarlo con PLE y luego resolverlo con un solver de PLE de propósito general, como CLEX. De este enofque nos ocuparemos a lo largo del notebook, pero tener presenten que existen muchos otros enfoques (backtracking, constraint programming, etc.). De todas formas, siempre que nos enfrentemos a un problema nuevo, es recomendable formularlo como PLE, ganaremos un mayor entendimiento del mismo y quizás se revelen detalles sobre su estructura.

En general, un mismo problema puede ser formulado como PLE de diferentes formas, y las reinas no son una excepción. Una de las formulaciones más sencillas surge de observar la siguiente condición suficiente y necesaria para ser solución: <i>el número de reinas en cada fila/columna/diagonal debe ser a lo sumo uno.</i>

#### Variables

Para simplificar la redacción, notaremos $\mathcal{N} = \{0\ldots,N-1\}$. Para cada $(i,j) \in \mathcal N \times \mathcal N$, definimos una variable binaria
$$ x_{i,j} = \begin{cases} \displaystyle
  1 & \text{ si hay una reina en la casilla } (i,j), \\
  0 & \text{ en caso contrario.}
\end{cases}$$
En este caso, $x_{0,0}$ se corresponde con la casilla "a1", $x_{0,1}$ con "b1", y así sucesivamente. Esta simplificación es necesaria porque en programación los arreglos están indexados con números naturales comenzando desde el 0.

#### Función objetivo
Debemos maximizar el número de reinas colocadas, es decir:
$$\max \sum_{(i,j) \in \mathcal N \times \mathcal N} x_{i,j} $$

#### Restricciones

* En cada fila hay a lo sumo una reina:
$$ \sum_{j \in \mathcal N} x_{i,j} \leq 1,~~~ \forall\ i \in \mathcal N. $$

* En cada columna hay a lo sumo una reina:
$$ \sum_{i \in \mathcal N} x_{i,j} \leq 1,~~~ \forall\ j \in \mathcal N. $$

* En cada diagonal hay a lo sumo una reina. Para no complicar demasiado estas restricciones, supongamos que $\mathcal D$ es el conjunto de todas las diagonales de largo al menos 2. Es decir, cada $D \in \mathcal D$ contiene todas las casillas que se encuentran en una misma diagonal. Por ejemplo, la diagonal a 45º que inicia en "a1" es $\{(k,k): k \in \mathcal N\}$, la inmediata superior que inicia en "a2" es $\{(1+k,k): k \in \mathcal N \setminus \{7\} \}$, y así sucesivamente. Luego podemos escribir estas restricciones como:
$$ \sum_{(i,j) \in D} x_{i,j} \leq 1,~~~ \forall\ D \in \mathcal D. $$
Notar que no consideramos diagonales con una única casilla porque dichas restricciones están implicadas por la binaridad de las variables.

En total hay $N^2$ variables y $6(N-1)$ restricciones.

### Modelado en python

Una vez escrito el modelo en lápiz y papel, lo siguiente es pasarlo a python. 

Los primeros pasos son de inicialización y nunca deben faltar. Comenzamos importando los paquetes necesarios para llamar a CPLEX.

In [ ]:
import cplex
from docplex.mp.model import Model

A continuación llamamos al constructor de la clase <code>Model</code> para construir un objeto que contendrá toda la información de nuestro modelo. En este paso podemos especificar un nombre para nuestro modelo mediante el atributo de clase <code>name</code>. Finalmente guardamos el modelo en una variable de python (no confundir con variable del modelo) de nombre <code>modelo</code> para referenciarlo más adelante. 

In [ ]:
modelo = Model(name='queens')

Lo siguiente es empezar a poblar el modelo con las variables, la función objetivo y las restricciones. Comencemos 
por las variables.

#### Variables

Primero vamos a definir variables de python para almacenar a $N$ y $\mathcal N$:

In [ ]:
N = 8 # 8 reinas
NN = range(8) ## NN = {0,...,7} 

Vamos a guardar en una variable de python de nombre <code>X</code> una matriz de tamaño $N \times N$. Inicialmente esta matriz estará vacía, es decir, rellena de 0's. Iterativamente la iremos poblando con las variables del modelo. El método <code>binary_var()</code> de la clase <code>Model</code> se usa para construir una variable del modelo. Aunque no es necesario, es recomendable darle un nombre a cada variable del modelo mediante el atributo <code>name</code>, lo que nos facilitará su interpretación una vez resuelto el modelo. En este caso, a la variable $x_{i,j}$ le daremos el nombre 'x_i_j'.

In [ ]:
X = [[0 for i in NN] for j in NN] # Matriz de tamaño NxN
# Rellenamos la matriz con las variables del modelo
for i in NN:
    for j in NN:
        X[i][j] = modelo.binary_var(name='x_' + str(i) + '_' + str(j))

Para los más experimentados en python, las 3 líneas de arriba se pueden escribir en una única, de la siguiente forma. Usamos <code>%%script true</code> al comienzo de la celda para impedir que jupyter la ejecute y vuelva a agregar las variables al modelo.

In [ ]:
%%script true

X = [[modelo.binary_var(name='x_'+ str(i) + '_' + str(j)) for i in NN] for j in NN]

Podemos verificar que en las casillas de <code>X</code> están efectivamente las variables del modelo. Por ejemplo en la (0,0) está $x_{0,0}$.

In [ ]:
print(X[0][0])

Antes de seguir, vale la pena hacer la siguiente aclaración. Es probable que si reejecutamos algunas de las celdas se dupliquen elementos en el modelo (por ejemplo la que construye la matriz <code>X</code>). Si esto sucede, debemos reiniciar el modelo: vamos a la pestaña "Kernel" y seleccionamos "Restart & Clear Output".

#### Función objetivo

Recordemos nuestra función objetivo:
$$\max \sum_{(i,j) \in \mathcal N \times \mathcal N} x_{i,j} $$

Veremos dos formas para escribirlo en python. Empecemos por la más natural. Vamos iterando sobre $(i,j) \in \mathcal N \times \mathcal N$ y acumulamos la suma de $X[i][j]$ en una variable de python de nombre <code>fobj</code> inicializada en 0.

In [ ]:
fobj = 0
for i in NN:
    for j in NN:
        fobj += X[i][j]

Para la segunda forma, usaremos el método <code>sum</code> de la clase <code>Model</code>, que nos permitirá escribir sumatorias en una única línea. Primero se escribe el término de la sumatoria y luego el rango de los índices.

In [ ]:
fobj = modelo.sum(X[i][j] for i in NN for j in NN)

Finalmente, indicamos que es un problema de maximización. Para ello, le pasamos la función objetivo <code>fobj</code> al modelo mediante el método <code>maximize</code> de la clase <code>Model</code>. De forma análoga, tenemos el método <code>minimize</code> para problemas de minimización.

In [ ]:
modelo.maximize(fobj)

#### Restricciones

Empecemos por las restricciones asociadas a las filas:
$$ \sum_{j \in \mathcal N} x_{i,j} \leq 1,~~~ \forall\ i \in \mathcal N. $$

Vamos de a poco. Primero necesitamos un bucle <code>for</code> para escribir el $\forall\ i \in \mathcal N$, es decir:
<p><code>for i in NN:
    ...
</code></p>

Ahora, en cada iteración del bucle tenemos que escribir la restricción del modelo. Vamos a aprovechar el método <code>sum</code> que aprendimos a user más arriba y además le indicamos que debe ser menor o igual a 1, es decir:
<code>modelo.sum(X[i][j] for j in NN) <= 1</code>
    
Finalmente, usamos el método <code>add_constraint</code> de la clase <code>Model</code> para agregar la restricción al modelo. Todo junto queda así:

In [ ]:
for i in NN:
    modelo.add_constraint(modelo.sum(X[i][j] for j in NN) <= 1)

Análogamente procedemos con las restricciones asociadas a las columnas.
$$ \sum_{i \in \mathcal N} x_{i,j} \leq 1,~~~ \forall\ j \in \mathcal N. $$

In [ ]:
for j in NN:
    modelo.add_constraint(modelo.sum(X[i][j] for i in NN) <= 1)

Resta proceder con las restricciones asociadas a las diagonales. 
$$ \sum_{(i,j) \in D} x_{i,j} \leq 1,~~~ \forall\ D \in \mathcal D. $$

Esta parte no es difícil, pero es algo engorrosa la generación del conjunto $\mathcal D$ de columnas. Hay muchas formas de hacer esto, veamos una posibilidad... Comenzamos definiendo una variable de pyhton de nombre <code>DD</code> con una lista inicialmente vacía e iterativamente iremos agregando todas las diagonales. Para generar las diagonales seguimos la siguinte idea.
1. La diagonal principal a 45 grados es [(0,0),(1,1),(2,2),(3,3),(4,4),(5,5),(6,6),(7,7)]. Esto lo escribimos de forma compacta en python como 
<p><code>[(j,j) for j in NN]</code></p><br>

2. La diagonal inmediata superior se obtiene a partir de la anterior sumando 1 a la primera componente de todos los pares y aplicando el módulo por $N$ por si la suma se sale del tablero. Es decir, nos queda la diagonal [(1,0),(2,1),(3,2),(4,3),(5,4),(6,5),(7,6),(0,7)]. En python lo escribimos como:
<p><code>D1 = [((j+1) % N, j) for j in NN]</code></p><br>

3. La diagonal del punto anterior en realidad no es continua, sino que son dos diagonales. Los primeros $N-1$ elementos son la diagonal [(1,0),(2,1),(3,2),(4,3),(5,4),(6,5),(7,6)] y el restante es la diagonal [(0,7)]. En python podemos escribir una lista que contenga a estas dos diagionales de la siguiente forma.
<p><code>[D1[:N-1], D1[N-1:]]</code></p><br>
Recordar que <code>D1[:N-1]</code> es la sublista de <code>D1</code> que tiene los elementos desde el índice 0 al $N-2$ y que <code>D1[N-1:]</code> es la sublista de <code>D1</code> que tiene los elementos desde el índice $N-1$ al último.


4. Agregamos cada una de estas diagionales a DD, siempre que tengan largo 2 o más:
<p><code>for A in [D1[:N-1],D1[N-1:]]:
        if len(A) >= 2:
            DD.append(A)
</code></p><br>
    
5. Análogamente, la diagonal inmediate superior se obtiene a partir del primer punto sumando 2 a la primera componente de todos los pares y aplicando el módulo por $N$. Y así sucesivamente hasta dar la vuelta completa... 

Juntando todo en un bucle <code>for</code>, donde <code>i</code> es el desplazamiento de la diagonal original, tenemos el siguiente fragmento de código:

In [ ]:
DD = []
for i in NN:
    D1 = [((j+i) % N, j) for j in NN]
    for A in [D1[:N-i],D1[N-i:]]:
        if len(A) >= 2:
            DD.append(A)

Análogamente, agregamos las diagonales a -45º, acomodando los índices de manera adecuada.

In [ ]:
for i in NN:
    D2 = [((N-1-j+i) % N, j) for j in NN]
    for A in [D2[:i],D2[i:]]:
        if len(A) >= 2:
            DD.append(A)

Ahora sí, ya estamos en condiciones de agregar las restricciones al modelo.

In [ ]:
for D in DD:
    modelo.add_constraint(modelo.sum(X[i][j] for (i,j) in D) <= 1)

Hemos terminado de construir el modelo en python. Podemos imprimir un resumen del modelo usando el método <code>print_information</code> de la clase <code>Model</code>:

In [ ]:
modelo.print_information()

### Resolución con CPLEX

Llegó la hora de resolver el modelo. Esta parte es muy sencilla, llamamos al método <code>solve</code> de la clase <code>Model</code>. Usamos el parámetro <code>log_output=True</code> para indicarle al solver que nos muestre el log por pantalla. Esté método retorna un objeto con información de la solución, el cual lo guardaremos en una variable de pyhton de nombre <code>sol</code>:

In [ ]:
sol = modelo.solve(log_output=True)

El log nos muestra mucha información...

* En la primer parte se encuentra toda la información referida al preprocesamiento, es decir, todo el trabajo que realiza el solver antes de empezar a resolver, por ejemplo: encontrar un incumbente (solución factible) de costo 0 (¡ja! qué inteligente) con sus propias heurísticas, fijar variables, eliminar restricciones implicadas, encontrar "cliques" para ser usadas posteriormente como planos de cortes, etc.

* Luego de eso, nos muestra información sobre el branch-and-cut. Las primeras iteraciones (Node 0) se realizan sobre el nodo raíz del árbol de branch-and-cut. Durante esas iteraciones el solver intenta encontrar soluciones factibles con sus propias heurísticas primales y también aplicar planos de corte para reforzar la relajación lineal. Vemos que el solver comienza con la cota superior 64 (que sale de sumar todas las restricciones de binaridad de las variables) y tras resolver la relajación lineal baja a 8. También vemos cómo el solver encuentra soluciones factibles (cotas inferiores) cada vez mejores con costos: 0; 5; 7; y finalmente 8 donde coincide con el valor de la relajación lineal y el algoritmo termina. Observar que en este caso, el valor óptimo de relajación lineal de la raíz coincide con el valor óptimo entero, pero no siempre será así. El solver mide qué tan cerca están sus cotas inferior y superior a través del gap. Cuando el gap llega a 0% quiere decir que las cotas coinciden y el algoritmo termina. De no cerrar el gap en el nodo raíz, el solver empieza a branchear (lo veremos en los siguientes notebooks).

* Finalmente nos reporta algunas estadísticas finales. Por ejemplo, cuántos planos de corte se agregaron (en este caso no se reportan porque no fueron necesarios) y el tiempo de ejecución.
 
Para ver la solución podemos usar el método <code>display</code> de la clase <code>SolveSolution</code>. Aquí es donde vemos la importancia de haberle puesto nombre a las variables.

In [ ]:
sol.display()

Como comodín podemos ver gráficamente esta solución sobre el tablero usando el paquete <code>chess</code> y corroborar visualmente que la solución es correcta. De nuevo, no es necesario entender el siguiente código, simplemente hay que ejecutarlo. 

In [ ]:
board = chess.Board()
board.clear()
for i in NN:
    for j in NN:
        if sol.get_value(X[i][j]) > 0.5:
            squareIndex=i*N+j
            board.set_piece_at(chess.SQUARES[squareIndex], chess.Piece(chess.QUEEN, chess.WHITE))
board

¡Felicitaciones, has conseguido resolver el problema! &#x1f389;&#x1f37e;&#x1f973;.

A modo de completitud dejamos en una única celda todo el código escrito, con sus respectivos comentarios para documentarlo.

In [ ]:
# Importamos los paquetes de CPLEX 
import cplex
from docplex.mp.model import Model

# Inicializamos el modelo
modelo = Model(name='queens')

N = 8 # 8 reinas
NN = range(8) ## NN = {0,...,7} 

# Agregamos las variables al modelo
X = [[modelo.binary_var(name='x_'+ str(i) + '_' + str(j)) for i in NN] for j in NN]

# Agregamos la funcion objetivo al modelo
fobj = modelo.sum(X[i][j] for i in NN for j in NN)
modelo.maximize(fobj)

# Agregamos las restricciones de fila al modelo
for i in NN:
    modelo.add_constraint(modelo.sum(X[i][j] for j in NN) <= 1)

# Agregamos las restricciones de columna al modelo
for j in NN:
    modelo.add_constraint(modelo.sum(X[i][j] for i in NN) <= 1)

# Construimos el conjunto de diagonales
DD = []
for i in NN:
    D1 = [((j+i) % N, j) for j in NN]
    D2 = [((N-1-j+i) % N, j) for j in NN]
    for A in [D1[:N-i],D1[N-i:],D2[:i],D2[i:]]:
        if len(A) >= 2:
            DD.append(A)

# Agregamos las restricciones de diagonal al modelo
for D in DD:
    modelo.add_constraint(modelo.sum(X[i][j] for (i,j) in D) <= 1)

# Resumen del modelo 
modelo.print_information()

# Resolucion del modelo
sol = modelo.solve(log_output=True)

# Imprimir solucion
sol.display()

Para reforzar los conocimientos aprendidos te proponemos resolver las siguientes actividades.

### Actividades

1. Añada una restricción que obligue a colocar una reina en una posición fija del tablero. Las restricciones por igualdad se pueden agregar al modelo usando el operador <code>==</code> en lugar de <code><=</code> ¿Sigue siendo 8 la solución óptima? ¿Puede forzar a que sea inferior a ese número?

In [ ]:
### Espacio para completar la actividad 1.

2. Añada una/s restriccion/es al modelo que hagan que la instancia sea infactible. Analice la salida de CPLEX y haga un <code>display()</code> de la solución. ¿Qué se observa? ¿Aparece algún error en la ejecución? Más adelante veremos como manejar estos casos.

In [ ]:
### Espacio para completar la actividad 2.

3. Proponga una formulación alternativa para el problema de las $N$ reinas utilizando la siguiente idea para las restricciones: <i>"En todo par de casillas en conflicto (en una misma fila/columna/diagonal) puede haber a lo sumo una reina"</i>. Resuelva el modelo con CPLEX. ¿Observa alguna diferencia respecto al primer modelo? 

In [ ]:
# Puede ser de utilidad la siguiente función que determina
# si dos casillas (a,b) y (c,d) estan en conflicto:
def conflicto(a,b,c,d):
    if a == c: # Misma fila
        return True
    if b == d: # Misma columna
        return True
    if abs(a-c) == abs(b-d): # Misma diagonal
        return True
    return False

In [ ]:
### Espacio para completar la actividad 3.

### COMPLETAR

### Agregamos las restricciones al modelo
for a in NN:
    for b in NN:
        for c in NN:
            for d in NN:
                if ((a < c or (a == c and b < d)) # orden lexicografico
                and conflicto(a,b,c,d)): # hay conflicto
                    ### COMPLETAR: Agregar restriccion
                    
### COMPLETAR

4. Vamos con un ejercicio un poco más difícil. Utilizar todo lo aprendido para resolver el siguiente problema:
#### Problema de dominación con reinas
<i>Entrada:</i> Un número natural $N$.<br>
<i>Objetivo:</i> Ubicar la menor cantidad de reinas en las casillas de un tablero de ajedrez de tamaño $N \times N$ inicialmente vacío, de modo que todas las casillas estén atacadas por al menos una reina.

In [ ]:
### Espacio para completar la actividad 4.